### This Jupyter notebook provides examples of how the pipeline metadata stored with CMF tracking layer can be queried with CMF query layer.

In [1]:
import pandas as pd
from cmflib import cmfquery

##### Initialize the library and get all the stages in the pipeline
Point the library to the metadata file. <br>
The `get_pipeline_stages` call point to the different stages in the pipeline.

In [ ]:
query = cmfquery.CmfQuery("./mlmd")
stages = query.get_pipeline_stages("Test-env")
print(stages)

##### Query the Executions in each stage

In [ ]:
executions = query.get_all_executions_in_stage('Prepare')
print(executions)

##### Query the Executions in each stage


In [ ]:
executions = query.get_all_executions_in_stage('Featurize')
print(executions)

##### Query the Executions in each stage

In [ ]:
executions = query.get_all_executions_in_stage('Train')
print(executions)

##### Query the Executions in each stage

In [ ]:
executions = query.get_all_executions_in_stage('Evaluate')
print(executions)

##### Get all the artifacts of execution. 
<b>input parameter - execution_id</b><br>
<b>output parameter - artifacts</b><br>


In [ ]:
artifacts = query.get_all_artifacts_for_execution(1)
print(artifacts)

In [ ]:
# print(pd.options.display.max_colwidth)
pd.options.display.max_colwidth = 75
artifacts = query.get_all_artifacts_for_execution(2)

# print(artifacts)
print(artifacts["name"])
print(artifacts["event"])

#### get all executions for an artifact(pass the artifact full name as the input parameter)

In [ ]:
linked = query.get_all_executions_for_artifact("artifacts/data.xml.gz:a304afb96060aad90176268345e10355")
print(linked)

#### Get all the parent artifacts of an artifact. Provides the artifact lineage chain

In [ ]:
linked = query.get_all_parent_artifacts("artifacts/features/test.pkl")
print(linked)

#### Get all the child artifacts of an artifact. Provides the lineage chain in the downstream direction

In [ ]:
linked = query.get_all_child_artifacts("artifacts/features/train.pkl")

print("Name : " + linked["name"].to_string(index=False, header=False))
print("Type : " + linked["type"].to_string(index=False, header=False))
print("URI : " + linked["uri"].to_string(index=False, header=False))

#### Get all the parent artifacts of an artifact. Provides the artifact lineage chain

In [ ]:
linked = query.get_all_parent_artifacts("artifacts/model/model.pkl")
print("NAME")
print(linked["name"].to_string(index=False, header=False))
print("TYPE")
print(linked["type"].to_string(index=False, header=False))
print("URI")
print(linked["uri"].to_string(index=False, header=False))

In [ ]:
linked = query.get_all_parent_artifacts("artifacts/parsed/test.tsv")
print("Name : " + linked["name"].to_string(index=False, header=False))
print("Type : " + linked["type"].to_string(index=False, header=False))
print("URI : " + linked["uri"].to_string(index=False, header=False))

In [ ]:
linked = query.get_all_child_artifacts("artifacts/parsed/test.tsv")
print("NAME")
print(linked["name"].to_string(index=False, header=False))
print("TYPE")
print(linked["type"].to_string(index=False, header=False))
print("URI")
print(linked["uri"].to_string(index=False, header=False))

#### Get immediate child artifacts of an artifact. 

In [ ]:
linked = query.get_one_hop_child_artifacts("artifacts/data.xml.gz")
print("NAME")
print(linked["name"].to_string(index=False, header=False))
print("TYPE")
print(linked["type"].to_string(index=False, header=False))
print("URI")
print(linked["uri"].to_string(index=False, header=False))

In [ ]:
linked = query.get_all_child_artifacts("artifacts/data.xml.gz")
#print(linked.sort_values('create_time_since_epoch', ascending=True))
print("NAME")
print(linked["name"].to_string(index=False, header=False))
print("TYPE")
print(linked["type"].to_string(index=False, header=False))
print("URI")
print(linked["uri"].to_string(index=False, header=False))

In [ ]:
linked = query.get_all_artifacts_for_execution(4)
print("NAME")
print(linked["name"].to_string(index=False, header=False))
print("TYPE")
print(linked["type"].to_string(index=False, header=False))
print("URI")
print(linked["uri"].to_string(index=False, header=False))

### Change the metrics name in the get_artifact call with the metrics name from output of the previous cell

In [ ]:
artifacts = query.get_artifact("metrics:aaae534e-915d-11ec-b106-89841b9859cd:4")
print(artifacts)


In [ ]:
new_parquet_df = pd.read_parquet("./slice-a")
print(new_parquet_df)